# MIDI Piano Beats Generator (ver 1.0)

***

## Simple MIDI Performance Piano Beats Generator

***

### Project Los Angeles
### Tegridy Code 2021

***

# Setup environment

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools
!pip install tqdm

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
from datetime import datetime
import secrets
import numpy as np
import matplotlib.pyplot as plt
import copy
import tqdm
from tqdm import auto

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')
import TMIDIX

from IPython.display import display, Javascript, HTML, Audio

from google.colab import output, drive

os.chdir('/content/')
print('Loading complete. Enjoy! :)')

# Process and plot your source MIDI file

In [ ]:
#@title Process your source MIDI file here
#@markdown NOTE: For now everything is hard-coded to a single-track, MIDI channel 0, Piano-only MIDIs.

full_path_to_source_MIDI_file = "/content/tegridy-tools/tegridy-tools/seed2.mid" #@param {type:"string"}

desired_MIDI_channel_to_process = 0

encode_velocities = False
encode_MIDI_channels = False
add_transposed_dataset_by_this_many_pitches = 0
add_transposed_and_flipped_dataset = False
chordify_input_MIDIs = False
melody_conditioned_chords = False
melody_pitch_baseline = 60
time_denominator = 1
transform_to_pitch = 0
perfect_timings = True
MuseNet_encoding = True
chars_encoding_offset =  33

print('TMIDI Optimus MIDI Processor')
print('Starting up...')
###########

average_note_pitch = 0
min_note = 127
max_note = 0

files_count = 0

gfiles = 0

chords_list_f = []
melody_list_f = []

chords_list = []
chords_count = 0

melody_chords = []
melody_count = 0

TXT_String = ''

TXT = ''
melody = []
chords = []

###########

print('Loading MIDI file...')

filez = [full_path_to_source_MIDI_file]


for f in tqdm.auto.tqdm(filez):
  try:
    fn = os.path.basename(f)
    fn1 = fn.split('.')[0]

    files_count += 1
    TXT, melody, chords, bass_melody, karaokez, INTS, aux1, aux2 = TMIDIX.Optimus_MIDI_TXT_Processor(f, chordify_TXT=chordify_input_MIDIs, output_MIDI_channels=encode_MIDI_channels, char_offset=chars_encoding_offset, dataset_MIDI_events_time_denominator=time_denominator, output_velocity=encode_velocities, MIDI_channel=desired_MIDI_channel_to_process, MIDI_patch=range(0, 127), melody_conditioned_encoding=melody_conditioned_chords, melody_pitch_baseline=melody_pitch_baseline, perfect_timings=perfect_timings, musenet_encoding=MuseNet_encoding, transform=transform_to_pitch)
    TXT_String += TXT
    melody_list_f += melody
    chords_list_f += chords
    gfiles += 1

  except KeyboardInterrupt:
    print('Saving current progress and quitting...')
    break  
  
  except:
    print('Bad MIDI:', f)
    continue

TXT_String += 'TOTAL_SONGS_IN_DATASET=' + str(gfiles)

try:
  print('Task complete :)')
  print('==================================================')
  if add_transposed_dataset_by_this_many_pitches != 0:
    print('NOTE: Transposed dataset was added per users request.')
    print('==================================================')
  if add_transposed_and_flipped_dataset == True:
    print('NOTE: Flipped dataset was added per users request.')  
    print('==================================================')
  print('Number of processed dataset MIDI files:', files_count)
  print('Number of MIDI chords recorded:', len(chords_list_f))
  print('First chord event:', chords_list_f[0], 'Last chord event:', chords_list_f[-1]) 
  print('Number of recorded melody events:', len(melody_list_f))
  print('First melody event:', melody_list_f[0], 'Last Melody event:', melody_list_f[-1])
  print('Total number of MIDI events recorded:', len(chords_list_f) + len(melody_list_f))
  print('==================================================')

except:
  print('=' * 70)
  print('IO Error!')
  print('Please check that Dataset dir is not empty/check other IO code.')
  print('=' * 70)
  print('Shutting down...')
  print('=' * 70)

In [ ]:
#@title Create a 3D Scatter-plot of the processed MIDI file

chords_flat = []
st = []
du = []
pt = []

for c in chords_list_f:
  st.append(c[1])
  du.append(c[2])
  pt.append(c[4])

# Creating dataset
x1 = np.array(st)
y1 = np.array(du)
z1 = np.array(pt)

#z = np.random.randint(100, size =(50))
#x = np.random.randint(80, size =(50))
#y = np.random.randint(60, size =(50))
 
# Creating figure
fig = plt.figure(figsize = (15,12))
ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(x1, y1, z1, s = 10, c = z1)
#ax.set_position()
ax.set_xlabel('Start Times')
ax.set_ylabel('Durations')
ax.set_zlabel('Pitches')
plt.title(str(fn))
ax.view_init(60, 30)
# show plot
plt.show()

# Generate and download

In [ ]:
#@title Generate beats and download the results

#@markdown NOTE: Beats patterns are generated by a random factor so the results will be non-deterministic
min_beat_MIDI_patch_number = 44 #@param {type:"slider", min:0, max:100, step:1}
avg_beat_MIDI_patch_number = 41 #@param {type:"slider", min:0, max:100, step:1}
max_beat_MIDI_patch_number = 36 #@param {type:"slider", min:0, max:100, step:1}
beats_randomness_factor = 4 #@param {type:"slider", min:4, max:16, step:4}

print('=' * 70)
print('MIDI Piano Beats Generator')
print('Starting up...')
print('=' * 70)

print('Calculating beats...')
all_st_deltas = []
delta_sum = 0
min_beat = 510
avg_beat = 0
max_beat = 0

pe = copy.deepcopy(chords_list_f[0])

for c in chords_list_f[1:]:
  if abs(c[1] - pe[1]) > 0: min_beat = int(round(min(min_beat, abs(c[1] - pe[1])), -1))
  if abs(c[1] - pe[1]) <= 510: max_beat = int(round(max(max_beat, abs(c[1] - pe[1])), -2))
  all_st_deltas.append(abs(c[1] - pe[1]))
  delta_sum += abs(c[1] - pe[1])
  pe = copy.deepcopy(c)

avg_beat = int(round(delta_sum / len(all_st_deltas), -2))
print('=' * 70)

print('Min beat', min_beat, 'ms')
print('Avg beat', avg_beat, 'ms')
print('Max beat', max_beat, 'ms')
print('=' * 70)

print('Generating beats patterns...')
beats = []
song_f = []
mib = 0
avb = 0
mab = 0
for c in chords_list_f:
  if c[1] % min_beat == 0 and secrets.randbelow(beats_randomness_factor) == 0: 
    beats.append(['note', c[1], min_beat, 9, min_beat_MIDI_patch_number, max(c[5], 90)])
    mib += min_beat
  if c[1] % avg_beat != 0 and secrets.randbelow(beats_randomness_factor) == 0: 
    beats.append(['note', c[1], avg_beat, 9, avg_beat_MIDI_patch_number, max(c[5], 100)])
    avb += avg_beat
  if c[1] % max_beat != 0 and secrets.randbelow(beats_randomness_factor) == 0: 
    beats.append(['note', c[1], max_beat, 9, max_beat_MIDI_patch_number, max(c[5], 110)])
    mab += max_beat
print('=' * 70)

print('Finalizing the output...')
beats.sort()  
song_f.extend(chords_list_f)
song_f.extend(beats)
song_f.sort()
print('=' * 70)

TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f, output_file_name='/content/' + fn1 + '_with_beats' , number_of_ticks_per_quarter=500, output_signature='MIDI Piano Beats Generator', track_name='Song:' + fn1)
print('=' * 70)

print('Downloading resulting composition now...')
from google.colab import files
files.download('/content/' + fn1 + '_with_beats.mid')
print('=' * 70)

# Congrats! You did it! :)